In [ ]:
import ollama
import duckdb
import os
from duckdb_utils import generate_sql
from langchain_community.llms import Ollama

In [ ]:
file = 'salaries.db'
file2 = 'salaries.db.wal'

if os.path.exists(file):
    os.remove(file)
    os.remove(file2)

# Instantiate a DuckDB connection
connection = duckdb.connect('salaries.db')
# delete table if it exists in duckdb
connection.execute("DROP TABLE IF EXISTS salary")
# Read the CSV file
connection.execute("CREATE TABLE salary AS SELECT * FROM read_csv('data/salaries.csv', AUTO_DETECT=TRUE)")


In [ ]:
questions = [
            "get average salary by experience level",
            "Group average salary by job_title where company_location is US?",
            "Average salary by job titles? Make sure the output is sorted in descending order.",
            "Highest paid job title where company_location is US? Show the job title and the salary.",
            "Top 10 highest paid job titles where company_locationin is US? Show the job titles and the salaries.",
            ]

In [ ]:
# lang_llm = Ollama(model="mistral")
lang_llm = Ollama(model="duckdb-nsql:7b-q8_0", temperature=0.1)

# Using the ollama client directly from ollama package
# llm = ollama

# Using the ollama client from langchain_community package
llm = lang_llm

sql_query = []
# get the query for each question
for question in questions:
    sql_query.append(generate_sql(question, connection, ollama))
# loop through the queries and execute them together with the question
for i in range(len(questions)):
    print(f"Question: {questions[i]} \n")
    print(f"Query: {sql_query[i]} \n")
    print(connection.execute(sql_query[i]).fetchdf())
    print("\n\n")

In [ ]:
connection.close()
# remove salaries.db
!rm salaries.db*